In [20]:
import pandas as pd
import numpy as np
import os

In [35]:
import re
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
from collections import defaultdict
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import ngrams
import string
from collections import defaultdict
from nltk.stem.porter import PorterStemmer
from rank_bm25 import BM25Okapi
from sklearn.metrics import ndcg_score
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from nltk.util import bigrams
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
import string

nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:992)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:992)>


False

In [36]:
columns = ['note','auteur', 'avis','assureur', 'produit', 'type', 'date_publication', 'date_exp', 'avis_en', 'avis_cor', 'avis_cor_en']
df = pd.DataFrame(columns=columns)

In [37]:
folder_path = "Traduction_avis_clients"  # Specify the folder path

# Get the current working directory
script_directory = os.getcwd()

# Create the full path to the folder
folder_full_path = os.path.join(script_directory, folder_path)

# Check if the folder exists
if os.path.exists(folder_full_path) and os.path.isdir(folder_full_path):
    # List all files in the folder
    files = os.listdir(folder_full_path)

    # Loop through each file
    for file_name in files:
        # Create the full path to the file
        file_path = os.path.join(folder_full_path, file_name)

        # Check if it's a file (not a subdirectory)
        if os.path.isfile(file_path):
            # Read the content of the file
            try:
                small_df=pd.read_excel(file_path)
                try:
                    df = pd.concat([df, small_df], ignore_index=True)
                except:
                    print('unable to read data from '+file_name)
            except:
                print("unable to open file "+file_name)
else:
    print(f"The specified folder '{folder_path}' does not exist.")


unable to open file ~$avis_2_traduit.xlsx
unable to open file ~$avis_1_traduit.xlsx


In [38]:
df.dropna(subset=['avis', 'avis_en'], inplace=True)

In [39]:
def clean_sentence(val):
    """ Cette fonction permet de supprimer les lettres, nombres et ensuite on
    supprimer les stopwords et les mots de taille inférieure à 2 """
    STOP_WORDS = stopwords.words('english')
    STOP_WORDS.append("")
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = sentence.split(" ")

    for word in list(sentence):
        if (word in STOP_WORDS) | (len(word)<=2):
            sentence.remove(word)
    return sentence


## Cette tokenisation ne fonctionne pas très bien
def text2TokenList(text):
    # Utilisation de clean_sentence
    cleaned_text = clean_sentence(text)
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    token_list = [lemmatizer.lemmatize(token) for token in cleaned_text]
    return token_list

In [48]:
def text2BigramList(text):
    cleaned_text = clean_sentence(text)
    lemmatizer = WordNetLemmatizer()
    token_list = [lemmatizer.lemmatize(token) for token in cleaned_text]
    bigram_list=[]
    # Generate bigrams
    bigram_list += [f'{token1}_{token2}' for token1, token2 in bigrams(token_list)]
    
    return bigram_list

In [49]:
df['tokens'] = df['avis_en'].apply(text2TokenList)
df['bigrams'] = df['avis_en'].apply(text2BigramList)

In [42]:
df = df.drop(['auteur', 'avis_cor', 'avis_cor_en'], axis=1)

In [50]:
df.shape

(34434, 10)

In [51]:
df.head(5)

,note,avis,assureur,produit,type,date_publication,date_exp,avis_en,tokens,bigrams
0,NaN,j'ai quitté mon ancien contrat d'assurance che...,Néoliane Santé,sante,test,12/01/2017,01/01/2017,I left my former insurance contract at General...,"[left, former, insurance, contract, general, y...","[left_former, former_insurance, insurance_cont..."
1,NaN,j'ai souscrit à cette mutuelle l'année dernier...,Néoliane Santé,sante,test,09/01/2017,01/01/2017,I subscribed to this mutual a year last year a...,"[subscribed, mutual, year, last, year, stayed,...","[subscribed_mutual, mutual_year, year_last, la..."
2,NaN,"Impossible d'avoir le bon service , ils raccro...",Néoliane Santé,sante,test,24/11/2016,01/11/2016,"Impossible to have the right service, they han...","[impossible, right, service, hang, nose, know,...","[impossible_right, right_service, service_hang..."
3,NaN,Génération est une mutuelle très chère pour un...,Génération,sante,test,09/11/2021,01/11/2021,Generation is a very expensive mutual for a re...,"[generation, expensive, mutual, retiree, 150, ...","[generation_expensive, expensive_mutual, mutua..."
4,NaN,je viens d apprendre que je suis radié... j ap...,Génération,sante,test,08/11/2021,01/11/2021,I just learned that I am struck off ... I call...,"[learned, struck, call, adviser, give, answer,...","[learned_struck, struck_call, call_adviser, ad..."


In [52]:
df_train = df[df['type'] == 'train']
df_test=df[df['type'] == 'test']

In [53]:
df_train.to_csv('train_set.csv', index=False)
df_test.to_csv('test_set.csv', index=False)

In [55]:
from collections import Counter

all_tokens = [token for sublist in df['tokens'] for token in sublist]

token_counts = Counter(all_tokens)

most_common_token = token_counts.most_common(10)
print("Most Common Token:", most_common_token)

Most Common Token: [('insurance', 17048), ('service', 10669), ('price', 10630), ('year', 9094), ('contract', 8245), ('satisfied', 7968), ('customer', 6148), ('month', 5851), ('good', 5116), ('time', 5031)]


In [57]:
all_bigrams = [bigram for sublist in df['bigrams'] for bigram in sublist]

bigram_counts = Counter(all_bigrams)

most_common_bigram = bigram_counts.most_common(10)
print("Most Common Bigram:", most_common_bigram)

Most Common Bigram: [('satisfied_service', 2247), ('customer_service', 2166), ('direct_insurance', 1625), ('satisfied_price', 892), ('insurance_company', 675), ('service_price', 619), ('take_care', 605), ('recommend_insurance', 602), ('price_suit', 599), ('car_insurance', 579)]
